In [2]:
import pandas as pd
import numpy as np

In [3]:
datasset_link = "https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis/data"
df= pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv")
df

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [4]:
df.rename(columns={
    "2401": "col1",
    "Borderlands": "col2",
    "Positive": "sentiment",
    "im getting on borderlands and i will murder you all ,": "text"
}, inplace=True)


In [5]:
df

,col1,col2,sentiment,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [6]:
# lets analyse col2
df["col2"].value_counts()

col2
Microsoft                            2400
MaddenNFL                            2400
TomClancysRainbowSix                 2400
LeagueOfLegends                      2394
CallOfDuty                           2394
Verizon                              2382
CallOfDutyBlackopsColdWar            2376
ApexLegends                          2376
Facebook                             2370
WorldOfCraft                         2364
Dota2                                2364
NBA2K                                2352
TomClancysGhostRecon                 2346
Battlefield                          2346
FIFA                                 2340
Overwatch                            2334
Xbox(Xseries)                        2334
johnson&johnson                      2328
Amazon                               2316
HomeDepot                            2310
PlayStation5(PS5)                    2310
CS-GO                                2304
Cyberpunk2077                        2304
GrandTheftAuto(GTA)          

In [7]:
# lets drop columns
df.drop(columns = ["col1","col2"],inplace = True)


In [8]:
df

,sentiment,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...
...,...,...
74676,Positive,Just realized that the Windows partition of my...
74677,Positive,Just realized that my Mac window partition is ...
74678,Positive,Just realized the windows partition of my Mac ...
74679,Positive,Just realized between the windows partition of...


In [9]:
df.isna().sum()

sentiment      0
text         686
dtype: int64

In [10]:
df.dropna(inplace = True)

In [11]:
# lets first preprcoss our datset little bit 
# lower case
# remove punctuation
# remove stop words 
# stemmimg 
# emoji handling
# complete sort forms 


In [12]:
import re
import string
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import emoji
import nltk
nltk.download('averaged_perceptron_tagger_eng')



stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Helper function to get POS tag for accurate lemmatization
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    from nltk import pos_tag
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Replace emojis with text description
    text = emoji.demojize(text)
    
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove stop words and apply lemmatization
    filtered_words = []
    words = text.split()
    for word in words:
        if word not in stop_words:
            pos = get_wordnet_pos(word)
            lemma = lemmatizer.lemmatize(word, pos)
            filtered_words.append(lemma)
    
    # Join words back to a single string
    processed_text = ' '.join(filtered_words)
    
    return processed_text


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [13]:
df["processed_text"] = df["text"].apply(preprocess_text)


In [14]:
df.drop(columns ="text",inplace = True)

In [15]:
# similarly lets import validation dataset for testing our model 
dftest = pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv")
dftest


,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...,...,...
994,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
995,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
996,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
997,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [16]:
# lets preprocess test dataset for modeling
dftest.rename(columns = {
    "Facebook":"col1",
    "Irrelevant":"sentiment",
    "I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣":"text" 
},inplace = True)

In [17]:
dftest

,3364,col1,sentiment,text
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...,...,...
994,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
995,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
996,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
997,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [18]:
dftest.drop(columns = ["3364","col1"],inplace = True)

In [19]:
dftest["text"] = dftest["text"].apply(preprocess_text)

In [20]:
dftest

,sentiment,text
0,Neutral,bbc news amazon bos jeff bezos reject claim co...
1,Negative,microsoft pay word function poorly samsungus c...
2,Negative,csgo matchmaking full closet hack truly awful ...
3,Neutral,president slap american face really commit unl...
4,Negative,hi eahelp i’ve madeleine mccann cellar past 13...
...,...,...
994,Irrelevant,star toronto art culture capital canada it’s w...
995,Irrelevant,actually good move tot bring viewer one people...
996,Positive,today suck it’s time drink wine n play borderl...
997,Positive,bought fraction microsoft today small win


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Define TF-IDF vectorizer (you can tune max_features, ngram_range etc. as needed)
tfidf = TfidfVectorizer(max_features=17000, ngram_range=(1,2))




In [22]:
# Fit TF-IDF on training text and transform train and test text
X_train = tfidf.fit_transform(df['processed_text'])
X_test = tfidf.transform(dftest['text'])



In [24]:
# Assuming your sentiment labels are in 'label' column in both dfs
y_train = df['sentiment']
y_test = dftest['sentiment']



In [25]:
# Initialize and train Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)





LogisticRegression(max_iter=1000)

In [26]:
# Predict on test data
y_pred = model.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9049049049049049
Classification Report:
               precision    recall  f1-score   support

  Irrelevant       0.90      0.89      0.90       171
    Negative       0.88      0.94      0.91       266
     Neutral       0.95      0.86      0.90       285
    Positive       0.89      0.92      0.91       277

    accuracy                           0.90       999
   macro avg       0.91      0.90      0.90       999
weighted avg       0.91      0.90      0.90       999



In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Create classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train
rf_model.fit(X_train, y_train)

# Predict
y_pred = rf_model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")



Accuracy: 0.9610


In [28]:
# lets test our models
# for logistic regression
txt = "Excited about the new movie release this weekend! Can't wait to watch it 🎬✨"
data = preprocess_text(txt)
print(f"test after preprocess :-{data}")
data = tfidf.transform([data])
model.predict(data)

test after preprocess :-excite new movie release weekend cant wait watch clapperboardsparkles


array(['Positive'], dtype=object)

In [29]:
# for random forest
txt = "Excited about the new movie release this weekend! Can't wait to watch it 🎬✨"
data = preprocess_text(txt)
print(f"test after preprocess :-{data}")
data = tfidf.transform([data])
rf_model.predict(data)


test after preprocess :-excite new movie release weekend cant wait watch clapperboardsparkles


array(['Positive'], dtype=object)